In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os
import requests
import json
from urllib.parse import urlparse

In [18]:
main_link = 'https://api-ssl.bitly.com/v4/'

In [28]:
def shorten_link(token, url):
  headers = {
    'Authorization' : f'Bearer {token}'
  }
  link = f'{main_link}bitlinks'
  body = {
       'long_url': url
  }
  response = requests.post(link,json = body, headers = headers)
  response.raise_for_status()
  return response.json()['link']

In [20]:
  def count_clicks(token, url):
    headers = {
      'Authorization' : f'Bearer {token}'
    }
    link = f'{main_link}bitlinks/{urlparse(url).netloc}{urlparse(url).path}/clicks/summary'
    requests.get(link, headers = headers).raise_for_status()
    return requests.get(link, headers = headers).json()['total_clicks']

In [21]:
def is_bitlink(url,token):
  headers = {
    'Authorization' : f'Bearer {token}'
  }
  link = f'{main_link}bitlinks/{urlparse(url).netloc}{urlparse(url).path}'
  return requests.get(link, headers = headers).ok

In [25]:
def main():
  load_dotenv()
  TOKEN = os.environ['TOKEN']
  url = input('Введите ссылку: ')
  try:
    response = requests.get(url)
    response.raise_for_status()
  except requests.exceptions.HTTPError:
    print('Недействительная ссылка')
  else:
    if is_bitlink(url,TOKEN):
      try:
        clicks_count = count_clicks(TOKEN,url)
      except requests.exceptions.HTTPError:
        print('Операция завершена с ошибкой')
      else:
        print(f'По Вашей ссылке перешли {clicks_count} раз(а)')
    else:
      try:
        bitlink = shorten_link(TOKEN,url)
      except requests.exceptions.HTTPError:
        print('Операция завершена с ошибкой')
      else:
        print(f'Битлинк: {bitlink}')

In [ ]:
if __name__ == '__main__':
    main()